## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
import os
weather_data_to_load = os.path.join("..", "Weather_Database", "WeatherPy_Database.csv")

city_data_df = pd.read_csv(weather_data_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Masunga,BW,-20.6246,27.4488,91.58,24,9,15.46,clear sky,2021-12-04 08:20:50
1,1,Saint-Philippe,RE,-21.3585,55.7679,88.11,73,12,13.76,few clouds,2021-12-04 08:20:50
2,2,Coihaique,CL,-45.5752,-72.0662,48.81,81,0,4.61,clear sky,2021-12-04 08:16:21
3,3,Kruisfontein,ZA,-34.0033,24.7314,65.88,60,100,10.92,overcast clouds,2021-12-04 08:20:50
4,4,Codrington,AU,-38.2667,141.9667,57.87,67,100,11.99,overcast clouds,2021-12-04 08:20:51


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
1,1,Saint-Philippe,RE,-21.3585,55.7679,88.11,73,12,13.76,few clouds,2021-12-04 08:20:50
13,13,Nosy Varika,MG,-20.5833,48.5333,82.60,65,39,8.23,scattered clouds,2021-12-04 08:20:54
16,16,Richards Bay,ZA,-28.7830,32.0377,80.62,64,100,7.58,overcast clouds,2021-12-04 08:20:55
17,17,Touros,BR,-5.1989,-35.4608,79.70,80,100,12.80,overcast clouds,2021-12-04 08:20:55
22,22,Hithadhoo,MV,-0.6000,73.0833,81.88,77,43,10.63,scattered clouds,2021-12-04 08:20:57
26,26,Jalingo,NG,8.8833,11.3667,86.86,24,14,3.69,few clouds,2021-12-04 08:20:58
28,28,Xuddur,SO,4.1213,43.8894,84.45,43,6,18.07,clear sky,2021-12-04 08:20:59
31,31,Khipro,PK,25.8285,69.3772,82.89,16,0,7.49,clear sky,2021-12-04 08:21:00
38,38,Mahebourg,MU,-20.4081,57.7000,86.29,62,40,14.97,scattered clouds,2021-12-04 08:21:03
41,41,Georgetown,MY,5.4112,100.3354,89.55,64,40,1.01,scattered clouds,2021-12-04 08:17:32


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                178
City                   178
Country                173
Lat                    178
Lng                    178
Max Temp               178
Humidity               178
Cloudiness             178
Wind Speed             178
Current Description    178
Date                   178
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,88.11,few clouds,-21.3585,55.7679,
13,Nosy Varika,MG,82.60,scattered clouds,-20.5833,48.5333,
16,Richards Bay,ZA,80.62,overcast clouds,-28.7830,32.0377,
17,Touros,BR,79.70,overcast clouds,-5.1989,-35.4608,
22,Hithadhoo,MV,81.88,scattered clouds,-0.6000,73.0833,
26,Jalingo,NG,86.86,few clouds,8.8833,11.3667,
28,Xuddur,SO,84.45,clear sky,4.1213,43.8894,
31,Khipro,PK,82.89,clear sky,25.8285,69.3772,
38,Mahebourg,MU,86.29,scattered clouds,-20.4081,57.7000,
41,Georgetown,MY,89.55,scattered clouds,5.4112,100.3354,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping.")        

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] !=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Philippe,RE,88.11,few clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
13,Nosy Varika,MG,82.60,scattered clouds,-20.5833,48.5333,Hotel Volazara
16,Richards Bay,ZA,80.62,overcast clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
17,Touros,BR,79.70,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
22,Hithadhoo,MV,81.88,scattered clouds,-0.6000,73.0833,Scoop Guest House
...,...,...,...,...,...,...,...
668,Betul,IN,80.89,clear sky,21.9153,77.8961,STARiHotel Betul MP
673,Saint-Joseph,RE,87.10,few clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
674,Soubre,CI,81.32,overcast clouds,5.7856,-6.6083,Lynd Hotels Soubre
675,Ugoofaaru,MV,82.67,few clouds,5.6667,73.0000,Ungoofaaru Inn


In [34]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))